In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [5]:
from load_librairies import *
import xscale.signal.fitting as xsf
dir_hist_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/tos/atl3/'
dir_hist_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/uas/atl4/'
dir_rcp8_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/tos/atl3/'
dir_rcp8_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/uas/atl4/'

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'

import warnings
warnings.filterwarnings('ignore')


lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
def is_jja(month):
    return (month >= 6) & (month <= 8)

def is_mjj(month):
    return (month >= 5) & (month <= 7)


path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:
model = ['ACCESS1-0', 'ACCESS1-3','CMCC-CESM',
         'CMCC-CMS','CMCC-CM','CNRM-CM5','CSIRO-Mk3-6-0',
        'GFDL-CM3', 'GFDL-ESM2M','GISS-E2-H-CC','GISS-E2-H',
        'GISS-E2-R-CC','GISS-E2-R','HadGEM2-CC','IPSL-CM5A-LR',
        'IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM-CHEM','MIROC-ESM',
        'MIROC5','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']

# Historical

In [4]:
alpha_bf1_hist_jja = np.ones((len(model)))
alpha_bf1_hist = np.ones((len(model)))
alpha_bf1_hist_monthly = np.ones((len(model),12))

r_bf1_hist_jja = np.ones((len(model)))
r_bf1_hist = np.ones((len(model)))
r_bf1_hist_monthly = np.ones((len(model),12))

std_bf1_hist_jja = np.ones((len(model)))
std_bf1_hist = np.ones((len(model)))
std_bf1_hist_monthly = np.ones((len(model),12))


p_value_bf1_hist_jja = np.ones((len(model)))
p_value_bf1_hist = np.ones((len(model)))
p_value_bf1_hist_monthly = np.ones((len(model),12))



for i in range(0,len(model)):
    

    sst = xr.open_dataset(dir_hist_sst+'tos_Omon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl3.nc')

    try:

        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        sst['time'] = sst.indexes['time'].to_datetimeindex()
        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    sst = sst.tos.chunk(chunks=None)
    sst = xsf.detrend(sst[:],dim='time',type='linear')
    sst = sst.load()
    ssta, _ = Atools.ano_norm_t(sst)
    ssta_atl3 = ssta.mean(dim='lon').mean(dim='lat')

    
    uas = xr.open_dataset(dir_hist_uas+'uas_Amon_'+model[i]+'_historical_r1i1p1_187001-200012_1deg_atl4.nc')
    print(model[i])

    try:

        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    uas = uas.__xarray_dataarray_variable__.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa, _ = Atools.ano_norm_t(uas)
    uasa_atl4 = uasa.mean(dim='lon').mean(dim='lat')

    ssta_atl3_jja = ssta_atl3.sel(time=is_jja(ssta_atl3['time.month']))
    uasa_atl4_jja = uasa_atl4.sel(time=is_mjj(uasa_atl4['time.month']))
    
    
    alpha_bf1_hist[i], _, r_bf1_hist[i], p_value_bf1_hist[i], std_bf1_hist[i] = stats.linregress(
    ssta_atl3[:], uasa_atl4[:])


    alpha_bf1_hist_jja[i], _, r_bf1_hist_jja[i], p_value_bf1_hist_jja[i], std_bf1_hist_jja[i] = stats.linregress(
    ssta_atl3_jja[:], uasa_atl4_jja[:])
    
    for j in range(0,12):
        alpha_bf1_hist_monthly[i,j], _, r_bf1_hist_monthly[i,j], p_value_bf1_hist_monthly[i,j], std_bf1_hist_monthly[i,j] = stats.linregress(
    ssta_atl3.sel(time=ssta_atl3['time.month'] == j+1), uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1))
        
        

ACCESS1-0
ACCESS1-3
CMCC-CESM
CMCC-CMS
CMCC-CM
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GFDL-ESM2M
GISS-E2-H-CC
GISS-E2-H
GISS-E2-R-CC
GISS-E2-R
HadGEM2-CC
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM-CHEM
MIROC-ESM
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M


In [6]:
bf1_hist_monthly  = xr.Dataset({'alpha': (['product','month'],alpha_bf1_hist_monthly),
                       'r': (['product','month'],r_bf1_hist_monthly),
                       'p_value': (['product','month'],p_value_bf1_hist_monthly),
                       'stddev': (['product','month'],std_bf1_hist_monthly)}
                       ,coords={'product':model,
                                'month': np.arange(1,13)})

bf1_hist  = xr.Dataset({'alpha': (['product'],alpha_bf1_hist),
                       'r': (['product'],r_bf1_hist),
                       'p_value': (['product'],p_value_bf1_hist),
                       'stddev': (['product'],std_bf1_hist)}
                       ,coords={'product':model})

bf1_hist_jja  = xr.Dataset({'alpha': (['product'],alpha_bf1_hist_jja),
                       'r': (['product'],r_bf1_hist_jja),
                       'p_value': (['product'],p_value_bf1_hist_jja),
                       'stddev': (['product'],std_bf1_hist_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF1',
                                    'long_name': 'First Bjerknes feedback component',
                                    'units': '(m/s)/K',
                                    'model': 'CMIP5',
                                    'Scenario': 'HIST',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

In [7]:
bf1_hist_jja.to_netcdf(path_data+'bf1_hist_mjj_CMIP5.nc')

## RCP8.5

In [8]:




alpha_bf1_jja = np.ones((len(model)))
alpha_bf1 = np.ones((len(model)))
alpha_bf1_monthly = np.ones((len(model),12))

r_bf1_jja = np.ones((len(model)))
r_bf1 = np.ones((len(model)))
r_bf1_monthly = np.ones((len(model),12))

std_bf1_jja = np.ones((len(model)))
std_bf1 = np.ones((len(model)))
std_bf1_monthly = np.ones((len(model),12))


p_value_bf1_jja = np.ones((len(model)))
p_value_bf1 = np.ones((len(model)))
p_value_bf1_monthly = np.ones((len(model),12))

model_id = []

for i in range(0,len(model)):
    
    
    sst = xr.open_dataset(dir_rcp8_sst+'tos_Omon_'+model[i]+'_rcp85_r1i1p1_201001-210012_1deg_atl3.nc')
  
    print(model[i])
    try:

        sst = sst.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        sst['time'] = sst.indexes['time'].to_datetimeindex()
        sst = sst.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
    sst = sst.tos.chunk(chunks=None)
    sst = xsf.detrend(sst[:],dim='time',type='linear')
    sst = sst.load()
    ssta, _ = Atools.ano_norm_t(sst)
    ssta_atl3 = ssta.mean(dim='lon').mean(dim='lat')

    uas = xr.open_dataset(dir_rcp8_uas+'uas_Amon_'+model[i]+'_rcp85_r1i1p1_201001-210012_1deg_atl4.nc')

    try:

        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
    uas = uas.__xarray_dataarray_variable__.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa, _ = Atools.ano_norm_t(uas)
    uasa_atl4 = uasa.mean(dim='lon').mean(dim='lat')

    ssta_atl3_jja = ssta_atl3.sel(time=is_jja(ssta_atl3['time.month']))
    uasa_atl4_jja = uasa_atl4.sel(time=is_mjj(uasa_atl4['time.month']))
    
    
    alpha_bf1[i], _, r_bf1[i], p_value_bf1[i], std_bf1[i] = stats.linregress(
    ssta_atl3[:], uasa_atl4[:])


    alpha_bf1_jja[i], _, r_bf1_jja[i], p_value_bf1_jja[i], std_bf1_jja[i] = stats.linregress(
    ssta_atl3_jja[:], uasa_atl4_jja[:])
    
    for j in range(0,12):
        alpha_bf1_monthly[i,j], _, r_bf1_monthly[i,j], p_value_bf1_monthly[i,j], std_bf1_monthly[i,j] = stats.linregress(
    ssta_atl3.sel(time=ssta_atl3['time.month'] == j+1), uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1))
       

ACCESS1-0
ACCESS1-3
CMCC-CESM
CMCC-CMS
CMCC-CM
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GFDL-ESM2M
GISS-E2-H-CC
GISS-E2-H
GISS-E2-R-CC
GISS-E2-R
HadGEM2-CC
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM-CHEM
MIROC-ESM
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M


In [9]:
bf1_rcp8_monthly  = xr.Dataset({'alpha': (['product','month'],alpha_bf1_monthly),
                       'r': (['product','month'],r_bf1_monthly),
                       'p_value': (['product','month'],p_value_bf1_monthly),
                       'stddev': (['product','month'],std_bf1_monthly)}
                       ,coords={'product':model,
                               'month': np.arange(1,13)})

bf1_rcp8  = xr.Dataset({'alpha': (['product'],alpha_bf1),
                       'r': (['product'],r_bf1),
                       'p_value': (['product'],p_value_bf1),
                       'stddev': (['product'],std_bf1)}
                       ,coords={'product':model})

bf1_rcp8_jja  = xr.Dataset({'alpha': (['product'],alpha_bf1_jja),
                       'r': (['product'],r_bf1_jja),
                       'p_value': (['product'],p_value_bf1_jja),
                       'stddev': (['product'],std_bf1_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF1',
                                    'long_name': 'First Bjerknes feedback component',
                                    'units': '(m/s)/K',
                                    'model': 'CMIP5',
                                    'Scenario': 'RCP85',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

In [10]:
#bf1_rcp8_monthly.to_netcdf('/Volumes/Arthur_disk2/Data/CMIP6/tmp_data/bf1_ssp5_monthly_CMIP5.nc')
bf1_rcp8_jja.to_netcdf(path_data+'bf1_ssp5_mjj_CMIP5.nc')

In [11]:
bf1_rcp8_mean = bf1_rcp8.alpha.mean(dim='product')
bf1_hist_mean = bf1_hist.alpha.mean(dim='product')

bf1_rcp8_mean_jja = bf1_rcp8_jja.alpha.mean(dim='product')
bf1_hist_mean_jja = bf1_hist_jja.alpha.mean(dim='product')